In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

class Transformer:
    def __init__(self,spark):
        self.spark = spark
    
    def transform(self, inputDf):
        pass
class FirstTransform(Transformer):
    def __init__(self, spark):
        self.spark = spark

    def transform(self, inputDf):
        """
        Customer who have bought Airpods after buying an iphone
        """

        transactionInputDf = inputDf.get("transactionInputDf")

        print("transactionInputDF in transform")

        transactionInputDf.show()

        #Airpods after buying iphone

        windowSpec = Window.partitionBy("customer_id").orderBy("transaction_date")

        transformedDF = transactionInputDf.withColumn(
            "next_product_name",lead("product_name").over(windowSpec)
        )

        print("Airpods after buying iphone")
        transformedDF.orderBy("customer_id", "transaction_date", "product_name").show()

        filteredDF = transformedDF.filter(
            (col("product_name") == 'iphone') & (col("next_product_name") == 'Airpods')
        )

        filteredDF.orderBy("customer_id", "transaction_date", "product_name").show()

        customerInputDF = inputDf.get("customerInputDF")

        customerInputDF.show()

        joinDF = customerInputDF.join(
            broadcast(filteredDF),
            on = "customer_id"
        )

        print("JOINED DF")
        joinDF.select(
            "customer_id",
            "customer_name",
            "location"
        ).show()

        return filteredDF
        

        return transformedDF    